In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams["figure.figsize"]=(20,10)

In [2]:
df1=pd.read_csv("Bengaluru_House_Data.csv")
df1.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Bengaluru_House_Data.csv'

In [ ]:
df1.shape

In [ ]:
df1.groupby('area_type')['area_type'].agg('count')

In [ ]:
df2=df1.drop(['area_type','society','balcony','availability'],axis='columns')
df2.head()

In [ ]:
df2.isnull().sum()

In [ ]:
df3=df2.dropna()
df3.isnull().sum()

In [ ]:
df3.shape

In [ ]:
df3['size'].unique()

In [ ]:
df3['bhk']=df3['size'].apply(lambda x:(x.split(" ")[0]))

In [ ]:
df3.head()

In [ ]:
df3['bhk'].unique()

In [ ]:
    df3['bhk'] = df3['bhk'].astype(int)
    print("\nNew dtypes after conversion:")
    print(df3.dtypes)

In [ ]:
df3[df3.bhk>20]

In [ ]:
df3.total_sqft.unique()

In [ ]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True    

In [ ]:
df3[~df3['total_sqft'].apply(is_float)]

In [ ]:
def convert_sqft_to_num(x):
    tokens=x.split('-')
    if len(tokens)==2:
        return(float(tokens[0])+float(tokens[1]))/2
    try:
        return float(x)    
    except:
        return None

In [ ]:
convert_sqft_to_num("1234")

In [ ]:
df4=df3.copy()
df4['total_sqft']=df4['total_sqft'].apply(convert_sqft_to_num)
df4.head(3)

In [ ]:
df4.loc[30]

In [ ]:
df5=df4.copy()
df5['price_per_sqft']=df5['price']*100000/df5['total_sqft']
df5.head()

In [ ]:
len(df5.location.unique())

In [ ]:
df5.location=df5.location.apply(lambda x:x.strip())
location_stats=df5.groupby('location')['location'].agg('count').sort_values(ascending=False)
location_stats

In [ ]:
len(location_stats[location_stats<=10])

In [ ]:
location_stats_less_than_10=location_stats[location_stats<=10]
location_stats_less_than_10

In [ ]:
len(df5.location.unique())

In [ ]:
df5.location=df5.location.apply(lambda x:'other' if x in location_stats_less_than_10 else x)
len(df5.location.unique())

In [ ]:
df5.head(10)

In [ ]:
df5[df5.total_sqft/df5.bhk<300].head()

In [ ]:
df5.shape

In [ ]:
df6=df5[~(df5.total_sqft/df5.bhk<300)]
df6.shape

In [ ]:
df6.price_per_sqft.describe()

In [ ]:
def remove_pps_outliers(df):
    df_out=pd.DataFrame()
    for key,subdf in df.groupby('location'):
        m=np.mean(subdf.price_per_sqft)
        st=np.std(subdf.price_per_sqft)
        reduced_df=subdf[(subdf.price_per_sqft>(m-st)) & (subdf.price_per_sqft<=(m+st))]
        df_out=pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out   
df7=remove_pps_outliers(df6)   
df7.shape

In [ ]:
def plot_scatter_chart(df,location):
    bhk2=df[(df.location==location)&(df.bhk==2)]
    bhk3=df[(df.location==location)&(df.bhk==3)]
    matplotlib.rcParams['figure.figsize']=(15,20)
    plt.scatter(bhk2.total_sqft,bhk2.price,color='blue',label='2 bhk',s=50)
    plt.scatter(bhk3.total_sqft,bhk3.price,marker='+',color='green',label='3 bhk',s=50)
    plt.xlabel('Total sqrt Feet area')
    plt.ylabel('Price')
    plt.title(location)
    plt.legend()
plot_scatter_chart(df7,"Rajaji Nagar")    
    

In [ ]:
def remove_bhk_outliers(df):
    exclude_indices=np.array([])
    for location,location_df in df.groupby('location'):
        bhk_stats={}
        for bhk,bhk_df in location_df.groupby('bhk'):
            bhk_stats[bhk]={
                'mean':np.mean(bhk_df.price_per_sqft),
                'std':np.std(bhk_df.price_per_sqft),
                'count':bhk_df.shape[0]
            }
        for bhk,bhk_df in location_df.groupby('bhk'):
            stats=bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices=np.append(exclude_indices,bhk_df[bhk_df.price_per_sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index') 

df8=remove_bhk_outliers(df7)
df8.shape

In [ ]:
plot_scatter_chart(df8,"Rajaji Nagar")

In [ ]:
import matplotlib
matplotlib.rcParams["figure.figsize"]=(20,10)
plt.hist(df8.price_per_sqft,rwidth=0.8)
plt.xlabel("Price per square feet")
plt.ylabel("coount")

In [ ]:
df8.bath.unique()

In [ ]:
df8[df8.bath>10]

In [ ]:
plt.hist(df8.bath,rwidth=0.8)
plt.xlabel("Number of Bathrooms")
plt.ylabel("Count")

In [ ]:
df8[df8.bath>df8.bhk+2]

In [ ]:
df9=df8[df8.bath<df8.bhk+2]
df9.shape

In [ ]:
df10=df9.drop(['size','price_per_sqft'],axis='columns')
df10.head(3)

In [ ]:
dummies=pd.get_dummies(df10.location)
dummies.head(3)

In [ ]:
df11=pd.concat([df10,dummies.drop('other',axis='columns')],axis='columns')
df11.head(3)

In [ ]:
df12=df11.drop('location',axis='columns')
df12.head(2)

In [ ]:
for col in df12.select_dtypes(include='bool').columns:
    df12[col] = df12[col].astype(int)

df12.head(2)

In [ ]:
df12.shape

In [ ]:
x=df12.drop('price',axis='columns')
x.head()

In [ ]:
y=df12.price
y.head()

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y, test_size=0.2,random_state=10)

In [ ]:
from sklearn.linear_model import LinearRegression
lr_clf=LinearRegression()
lr_clf.fit(x_train,y_train)
lr_clf.score(x_test,y_test)

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
cv=ShuffleSplit(n_splits=5,test_size=0.2,random_state=0)
cross_val_score(LinearRegression(),x,y,cv=cv)

In [ ]:
from sklearn.model_selection import GridSearchCV, ShuffleSplit
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.tree import DecisionTreeRegressor
import pandas as pd

def find_best_model_using_gridsearch(x,y):
    algos={
        'linear_regression':{
            'model':LinearRegression(),
            'params':{
                'fit_intercept':[True,False]   # fixed normalize issue
            }
        },
        'lasso':{
            'model':Lasso(),
            'params':{
                'alpha':[1,2],
                'selection':['random','cyclic']   # fixed "cy" -> "cyclic"
            }
        },
        'decision_tree':{
            'model':DecisionTreeRegressor(),
            'params':{
                'criterion':['squared_error','friedman_mse'],  # fixed "mse"
                'splitter':['best','random']
            }
        }
    }
    scores=[]
    cv=ShuffleSplit(n_splits=5,test_size=0.2,random_state=0)
    for algo_name,config in algos.items():
        gs=GridSearchCV(config['model'],config['params'],cv=cv,return_train_score=False)
        gs.fit(x,y)
        scores.append({
            'model':algo_name,
            'best_score':gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores,columns=['model','best_score','best_params'])
find_best_model_using_gridsearch(x,y)    


In [ ]:
x.columns

In [ ]:
# def predict_price(location,sqft,bath,bhk):
#     loc_index=np.where(x.columns==location)[0][0]
#     x=np.zeroes(len(x.columns))
#     x[0]=sqft
#     x[1]=bath
#     x[2]=bhk
#     if loc_index>=0:
#         x[loc_index]=1
#     return lr_clf.predict([x])[0]

In [ ]:
def predict_price(location, sqft, bath, bhk):
    loc_index = np.where(x.columns == location)[0]
    
    # Feature vector
    x_vec = np.zeros(len(x.columns))
    x_vec[0] = sqft
    x_vec[1] = bath
    x_vec[2] = bhk
    
    if len(loc_index) > 0:
        x_vec[loc_index[0]] = 1
    
    # ✅ FIXED: reshape instead of wrapping in list
    return lr_clf.predict(x_vec.reshape(1, -1))[0]

In [ ]:
predict_price('1st Phase JP Nagar',1000,2,2)

In [ ]:
predict_price('1st Phase JP Nagar',1000,3,3)

In [ ]:
predict_price('Indira Nagar',1000,2,2)

In [ ]:
import pickle
with open('bangalore_home_prices_model.pickle','wb') as f:
    pickle.dump(lr_clf,f)

In [ ]:
import json
columns={
    'data_columns':[col.lower() for col in x.columns]
}
with open("columns.json","w") as f:
    f.write(json.dumps(columns))